In [31]:
import z3
from pathlib import Path
from datetime import datetime
from collections import defaultdict

def extract_pair(pair):
    n, x = pair.strip().split()
    return x, int(n)

def parse(line):
    ingredients, result = line.strip().split(" => ")
    ingredients = [extract_pair(pair) for pair in ingredients.split(", ")]
    return ingredients, [extract_pair(result)]

def get_reactions(text):
    reactions = []
    for line in text.strip().splitlines():
        ingredients, result = parse(line)
        reaction = defaultdict(int)
        for x, n in ingredients:
            reaction[x] = -n
        for x, n in result:
            reaction[x] = n
        reactions.append(reaction)
    return reactions

### Part 1

In [43]:
def solve(text):
    reactions = get_reactions(text)
    products = {k for r in reactions for k in r.keys()}

    s = z3.Solver()

    n_reactions = [z3.Int(str(i)) for i in range(len(reactions))]
    start_ore = z3.Int("ore")
    s.add(start_ore >= 0)
    
    # what we start with
    initial = defaultdict(int, {"ORE": start_ore})

    # all non-negative
    for n_i in n_reactions:
        s.add(n_i >= 0)

    amounts = {}
    for product in products:
        amounts[product] = initial[product] + sum(reactions[i][product] * n_reactions[i] for i in range(len(reactions)))
        s.add(amounts[product] >= 0)

    s.add(amounts["FUEL"] >= 1)

    result, model, i = None, None, 1
    while s.check() == z3.sat:
        result, model = s.model()[start_ore], s.model()
        s.add(start_ore < result)
        print(i, datetime.now().isoformat(), result)
        i += 1
    return result, model, i

In [33]:
text = """
10 ORE => 10 A
1 ORE => 1 B
7 A, 1 B => 1 C
7 A, 1 C => 1 D
7 A, 1 D => 1 E
7 A, 1 E => 1 FUEL
"""
result, model, i = solve(text)
assert result == 31

2 2019-12-14T19:01:13.093596 31


In [27]:
text = """
157 ORE => 5 NZVS
165 ORE => 6 DCFZ
44 XJWVT, 5 KHKGT, 1 QDVJ, 29 NZVS, 9 GPVTF, 48 HKGWZ => 1 FUEL
12 HKGWZ, 1 GPVTF, 8 PSHF => 9 QDVJ
179 ORE => 7 PSHF
177 ORE => 5 HKGWZ
7 DCFZ, 7 PSHF => 2 XJWVT
165 ORE => 2 GPVTF
3 DCFZ, 7 NZVS, 5 HKGWZ, 10 PSHF => 8 KHKGT
"""
result, model, i = solve(text)
assert result == 13312

In [28]:
text = """
2 VPVL, 7 FWMGM, 2 CXFTF, 11 MNCFX => 1 STKFG
17 NVRVD, 3 JNWZP => 8 VPVL
53 STKFG, 6 MNCFX, 46 VJHF, 81 HVMC, 68 CXFTF, 25 GNMV => 1 FUEL
22 VJHF, 37 MNCFX => 5 FWMGM
139 ORE => 4 NVRVD
144 ORE => 7 JNWZP
5 MNCFX, 7 RFSQX, 2 FWMGM, 2 VPVL, 19 CXFTF => 3 HVMC
5 VJHF, 7 MNCFX, 9 VPVL, 37 CXFTF => 6 GNMV
145 ORE => 6 MNCFX
1 NVRVD => 8 CXFTF
1 VJHF, 6 MNCFX => 4 RFSQX
176 ORE => 6 VJHF
"""
result, model, i = solve(text)
assert result == 180697

In [40]:
text = """
171 ORE => 8 CNZTR
7 ZLQW, 3 BMBT, 9 XCVML, 26 XMNCP, 1 WPTQ, 2 MZWV, 1 RJRHP => 4 PLWSL
114 ORE => 4 BHXH
14 VRPVC => 6 BMBT
6 BHXH, 18 KTJDG, 12 WPTQ, 7 PLWSL, 31 FHTLT, 37 ZDVW => 1 FUEL
6 WPTQ, 2 BMBT, 8 ZLQW, 18 KTJDG, 1 XMNCP, 6 MZWV, 1 RJRHP => 6 FHTLT
15 XDBXC, 2 LTCX, 1 VRPVC => 6 ZLQW
13 WPTQ, 10 LTCX, 3 RJRHP, 14 XMNCP, 2 MZWV, 1 ZLQW => 1 ZDVW
5 BMBT => 4 WPTQ
189 ORE => 9 KTJDG
1 MZWV, 17 XDBXC, 3 XCVML => 2 XMNCP
12 VRPVC, 27 CNZTR => 2 XDBXC
15 KTJDG, 12 BHXH => 5 XCVML
3 BHXH, 2 VRPVC => 7 MZWV
121 ORE => 7 VRPVC
7 XCVML => 6 RJRHP
5 BHXH, 4 VRPVC => 5 LTCX
"""
result, model, i = solve(text)
assert result == 2210736

2 2019-12-14T21:12:36.470880 2210736


In [42]:
%%time
result, model, i = solve(Path("input.txt").read_text())
result, model, i

2 2019-12-14T21:12:41.634085 1802672476
3 2019-12-14T21:12:41.971065 613303
4 2019-12-14T21:12:42.007518 613084
5 2019-12-14T21:12:42.126171 612880
CPU times: user 933 ms, sys: 12 ms, total: 945 ms
Wall time: 914 ms


(612880, [29 = 1,
  37 = 73,
  3 = 10,
  32 = 123,
  23 = 11,
  39 = 1,
  43 = 716,
  0 = 184,
  12 = 439,
  38 = 33,
  11 = 6,
  31 = 1,
  21 = 94,
  40 = 12,
  2 = 24,
  47 = 1,
  33 = 97,
  10 = 26,
  52 = 10,
  4 = 5,
  17 = 46,
  24 = 14,
  35 = 10,
  ore = 612880,
  30 = 11,
  6 = 19,
  19 = 12,
  14 = 1572,
  25 = 1,
  27 = 1,
  7 = 4,
  41 = 177,
  55 = 151,
  49 = 5,
  1 = 491,
  28 = 14,
  46 = 430,
  15 = 69,
  8 = 1,
  34 = 34,
  53 = 979,
  26 = 2,
  44 = 9,
  16 = 385,
  18 = 37,
  20 = 7,
  36 = 97,
  54 = 1,
  9 = 51,
  45 = 1,
  51 = 77,
  5 = 33,
  50 = 339,
  42 = 4124,
  22 = 302,
  48 = 1,
  13 = 1129], 5)